In [2]:
from rag.pdf import PDFRetrievalChain
from dotenv import load_dotenv
load_dotenv()

pdf = PDFRetrievalChain().create_chain()
pdf_retriever = pdf.retriever
pdf_chain = pdf.chain


In [3]:
question = "정보보호위원회 위원에 대해 알려줘."
search_results = pdf.process_search_results(question)

In [ ]:
search_results

In [17]:
question = "사내 근로복지 기금 사무지침에서 정한 문화체육활동비에 대해 알려줘."
question = "전북은행 직원이 체육문화활동비 얼마를 받는지 알려줘."
question = "사내근로복지기금사무지침에 대해 알려주고 그 내용에 문화체육활동비에 대해 알려줘."
question = "체육문화활동비는 50만원 아니야?"

pdf_chain.invoke({
    "question": question, 
    "chat_history": []
})



'죄송합니다. 제공해주신 전북은행 E2007 계약직 및 업무지원직 지침의 제 48 조에 따르면, 체육문화활동비 지급은 사내근로복지기금사무지침을 준용한다고만 명시되어 있습니다. 따라서 체육문화활동비가 50만원인지에 대한 정보는 해당 지침에서 확인할 수 없습니다. \n\n체육문화활동비의 정확한 금액은 사내근로복지기금사무지침을 확인하시거나, 전북은행 인사팀에 문의하시는 것이 정확한 정보를 얻는 방법입니다.\n'

In [ ]:
체육문화활동비는 50만원 아니야?

In [18]:
pdf_retriever.invoke("사내 근로복지 기금 사무지침에서 정한 문화체육활동비에 대해 알려줘.")

[Document(id='d85fc4d6-5fed-411c-b6d2-24708c1fde9d', metadata={'장': '제 7 장 복지후생', 'source': '../rag_data/jbb/규정/E2007 계약직 및 업무지원직 지침.md', 'page': 0, 'title': 'E2007 계약직 및 업무지원직 지침', '제목': '계약직 및 업무지원직 지침', '조': '제 48 조 (체육문화활동비 지급)'}, page_content='#### 제 48 조 (체육문화활동비 지급)   직원(전문직직원 제외)의 체육문화활동비 지급은 사내근로복지기금사무지침을 준용한다.\n\n문서 : 전북은행 E2007 계약직 및 업무지원직 지침')]

In [23]:
from abc import ABC, abstractmethod
from operator import itemgetter

from langchain_core.prompts import load_prompt
from langchain_core.output_parsers import StrOutputParser
# from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_ollama import ChatOllama
from langchain import hub
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_chroma import Chroma
from langchain_core.prompts import load_prompt
from urllib.parse import urlencode

In [26]:
k = 3
model_name = "gemma3:12b"
ollama_url = "http://localhost:11434"
model_path = "/Users/netager/Docker_Data/openwebui-dify/rag_data/HUGGING_FACE_MODEL/BAAI_bge-m3"
chroma_path = "/Users/netager/Docker_Data/openwebui-dify/rag_data/Chroma_DB_NEW/chroma_bank_law_db"
prompt_path = "/Users/netager/Docker_Data/openwebui-dify/rag_data/prompts/law.yaml"

def create_embedding():
    return HuggingFaceEmbeddings(
        model_name= model_path,
        model_kwargs={"device": "mps"},  # cpu : 'cpu', macOS: 'mps', CUDA: 'cuda'
        encode_kwargs={"normalize_embeddings": True},
    )

def create_retriever(vectorstore):
    return vectorstore.as_retriever(
        search_type="similarity", 
        search_kwargs={"k": k},
        # Add mmr search for diversity
        # search_type="mmr",
        # search_kwargs={"k": 1, "fetch_k": 3, "lambda_mult": 0.5}
    )

def create_vectorstore():
    return Chroma(
            persist_directory=chroma_path,
            embedding_function=create_embedding(),
            collection_name="bank_law_case",
        )

In [30]:
# vectorstore = create_vectorstore()

def create_retriever(vectorstore):
    return vectorstore.as_retriever(
        search_type="similarity", 
        search_kwargs={"k": 5},
        # Add mmr search for diversity
        # search_type="mmr",
        # search_kwargs={"k": 1, "fetch_k": 3, "lambda_mult": 0.5}
    )

retriever = create_retriever(vectorstore)

In [35]:
retriever.invoke("사내근로복지기금의 수행 사업에 대해 알려줘.")

[Document(id='1af05650-fc16-4902-bf5a-31e98d2353ca', metadata={'page': 0, 'title': 'F1003 사내근로복지기금 정관', '장': '제 1 장 총 칙', '제목': '사내 근로복지기금 정관', '조': '제 5 조 (사업)', 'source': '../rag_data/jbb/규정/F1003 사내근로복지기금 정관.md'}, page_content='#### 제 5 조 (사업)   ① 기금은 제2조의 목적을 달성하기 위하여 다음의 사업을 행한다.\n1. 임 · 직원에 관련된 장학사업 2. 기타 사내근로복지기금법에 정한 범위 내에서 기금 협의회가 정하는 사업 ② 전항의 사업은 기금의 운영수익금으로 수행한다. 다만, 조성된 기금의 총액이 은행 자본금의 100분의 50을 초과하는 경우 그 초과액에 한 하여는 협의회가 의결한 기금 사용율 범위내에서 목적사업에 충당할 수 있다.\n\n문서 : 전북은행 F1003 사내근로복지기금 정관'),
 Document(id='4b30fc3b-40a2-4b4a-bc7a-525610f9eb2c', metadata={'source': '../rag_data/jbb/규정/F1003 사내근로복지기금 정관.md', 'title': 'F1003 사내근로복지기금 정관', 'page': 0}, page_content='전북은행 F1003 사내근로복지기금 정관 알려줘'),
 Document(id='7a5490ba-eac4-4612-907e-cf06d456fee2', metadata={'page': 0, 'title': 'F2006 사내근로복지기금 사무지침', 'source': '../rag_data/jbb/규정/F2006 사내근로복지기금 사무지침.md'}, page_content='전북은행 F2006 사내근로복지기금 사무지침 알려줘'),
 Document(id='eba2799d-52d0-4fc4-8005-8dc4ceeca8b4', metadata={'조': '제 1 조 (목적)', 'sou